<a href="https://colab.research.google.com/github/madhu1096/Recommendation-System-using-ML/blob/master/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie recommendation system

## Importing libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("/content/drive/My Drive/Machine Learning A-Z (Codes and Datasets)/Part 5 - Association Rule Learning/Section 28 - Apriori/Python/movie/tmdb_5000_credits.csv")
df2 = pd.read_csv("/content/drive/My Drive/Machine Learning A-Z (Codes and Datasets)/Part 5 - Association Rule Learning/Section 28 - Apriori/Python/movie/tmdb_5000_movies.csv")

In [ ]:
df1.head(5)

In [ ]:
df2.head(1)

## We need to extract these from the below 2 dataset's
['title', 'cast', 'director', 'keywords', 'genres']

In [5]:
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [ ]:
df2.head(2)

In [7]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
df2[feature].head(1)

In [9]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []
# Define new director, cast, genres and keywords features that are in a suitable form.
df2['director'] = df2['crew'].apply(get_director)

In [ ]:
features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(5)

In [12]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)


In [ ]:
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [14]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [15]:
df2['soup'].head(2)

0    cultureclash future spacewar samworthington zo...
1    ocean drugabuse exoticisland johnnydepp orland...
Name: soup, dtype: object

## Applying Count Vectorizer

In [16]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

## Creating Cosine Similarity between two Vectors

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

## Function to predict similar movies

In [18]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    print('Movie i have matched:'+title)
    idx = indices[title]
    print('Recommended Movies for you')
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:5]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

## Getting predictions

In [ ]:
get_recommendations('The Avengers', cosine_sim2)

## Soup is what we created in preprocessing step , then we do counter vectorizer, then we apply cosine similarity function to calculate similarities which will be usefull to get recommendations

In [ ]:
df2.head(2)